<a href="https://colab.research.google.com/github/MohammadAliAfsahi/YapAiTek-assignment/blob/master/YapAiTek_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#importing modules
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import losses
from keras import metrics


Using TensorFlow backend.


In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/YapAiTek assignment/data.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/YapAiTek assignment/test.csv')

In [6]:
data.head()

,Unnamed: 0,Episode,Station,Channel Type,Season,Year,Date,Day of week,Start_time,End_time,Length,Name of show,Name of episode,Genre,First time or rerun,# of episode in the season,Movie?,Game of the Canadiens during episode?,Market Share_total,Temperature in Montreal during episode
0,1,Vidéoclips V,V Total,General Channel,Fall,2016,2016-08-29,Monday,2016-08-29 06:00:00,2016-08-29 08:00:00,8,Vidéoclips V,NaN,Music Video Clips,No,Yes,No,No,0.9,20.400
1,2,Apollo dans l'frigo,V Total,General Channel,Fall,2016,2016-08-29,Monday,2016-08-29 08:00:00,2016-08-29 08:30:00,2,Apollo dans l'frigo,NaN,Informal Education and Recreation and Leisure,No,Yes,No,No,0.5,19.125
2,3,Infopublicité,V Total,General Channel,Fall,2016,2016-08-29,Monday,2016-08-29 08:30:00,2016-08-29 09:00:00,2,Infopublicité,NaN,"Infomercials, Promotional and Corporate Videos",No,Yes,No,No,0.3,19.125
3,4,"Infos, Les",V Total,General Channel,Fall,2016,2016-08-29,Monday,2016-08-29 09:00:00,2016-08-29 10:00:00,4,"Infos, Les",NaN,News,No,Yes,No,No,1.7,18.125
4,5,"Souper presque parfait, Un",V Total,General Channel,Fall,2016,2016-08-29,Monday,2016-08-29 10:00:00,2016-08-29 10:30:00,2,"Souper presque parfait, Un",NaN,Reality Programs,No,Yes,No,No,2.2,18.250


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616656 entries, 0 to 616655
Data columns (total 20 columns):
Unnamed: 0                                616656 non-null int64
Episode                                   616656 non-null object
Station                                   616656 non-null object
Channel Type                              616656 non-null object
Season                                    616656 non-null object
Year                                      616656 non-null int64
Date                                      616656 non-null object
Day of week                               616656 non-null object
Start_time                                616613 non-null object
End_time                                  616613 non-null object
Length                                    616656 non-null int64
Name of show                              616656 non-null object
Name of episode                           394077 non-null object
Genre                                     61665

In [8]:
data.isna().sum()

Unnamed: 0                                     0
Episode                                        0
Station                                        0
Channel Type                                   0
Season                                         0
Year                                           0
Date                                           0
Day of week                                    0
Start_time                                    43
End_time                                      43
Length                                         0
Name of show                                   0
Name of episode                           222579
Genre                                          0
First time or rerun                            0
# of episode in the season                     0
Movie?                                         0
Game of the Canadiens during episode?          0
Market Share_total                             0
Temperature in Montreal during episode     83344
dtype: int64

In [9]:
data['Market Share_total'].describe()

count    616656.000000
mean          3.011562
std           4.919562
min           0.000000
25%           0.400000
50%           1.400000
75%           3.400000
max          89.500000
Name: Market Share_total, dtype: float64

In [10]:
data['Station'].value_counts()

Teletoon Fr.+                          50750
Unis TV+                               46318
TQ Total                               44150
RDI+                                   40490
SRC Total                              37582
TV5+                                   29974
Z+                                     29715
Historia+                              29267
MusiquePlus+ (retired Aug 25, 2019)    27555
Canal Vie+                             27528
Canal D+                               26075
MAX+                                   24418
ARTV+                                  24298
VRAK+                                  23903
Evasion+                               23508
TVA Total                              21834
Series++                               21575
V Total                                21289
RDS2+                                  19075
LCN+                                   17536
RDS+                                   17468
Meteomedia+                            10524
PYC Online

In [11]:
data['Channel Type'].value_counts()

Specialty Channel    535951
General Channel       80705
Name: Channel Type, dtype: int64

In [12]:
data['Season'].value_counts()

Fall      234291
Winter    195257
Summer    127457
Spring     59651
Name: Season, dtype: int64

In [13]:
data['Year'].value_counts()

2018    260132
2017    255214
2016     75849
2019     25461
Name: Year, dtype: int64

In [14]:
data['Day of week'].value_counts()

Wednesday    89748
Tuesday      89058
Friday       88795
Monday       88597
Thursday     88507
Saturday     86604
Sunday       85347
Name: Day of week, dtype: int64

In [15]:
data['Length'].value_counts()

2     266631
4     170233
0      60086
1      55190
3      14709
8      13347
6       8766
12      4043
7       3542
28      3534
10      3160
16      2852
5       2251
84      1752
9       1557
14      1522
24       937
11       778
15       691
13       516
18       219
20       118
26        46
17        41
22        40
19        26
21        26
23        16
30         7
25         3
32         3
36         3
27         2
34         2
40         2
80         2
38         1
39         1
92         1
Name: Length, dtype: int64

In [16]:
data['Genre'].value_counts()

Animated Television Programs & Films                          88880
Ongoing Dramatic Series                                       79219
Reality Programs                                              72861
News                                                          71121
Long-form Documentary                                         50520
Informal Education and Recreation and Leisure                 44804
General Entertainment and Human Interest                      37275
Professional Sports                                           30871
Theatre Feature Films Aired on TV                             20682
Interstitial                                                  20000
Analysis and Interpretation                                   18353
Ongoing Comedy Series (Sitcoms)                               14366
Game Shows                                                    11809
Music Video Programs                                          11227
Infomercials, Promotional and Corporate Videos  

In [17]:
data['# of episode in the season'].value_counts()

Yes    606002
No      10654
Name: # of episode in the season, dtype: int64

In [18]:
# No Idea how it can help... 
data['Temperature in Montreal during episode'].describe()

count    533312.000000
mean          6.707428
std          12.178492
min         -27.425000
25%          -1.562500
50%           6.450000
75%          17.050000
max          34.950000
Name: Temperature in Montreal during episode, dtype: float64

In [19]:
data.columns

Index(['Unnamed: 0', 'Episode', 'Station', 'Channel Type', 'Season', 'Year',
       'Date', 'Day of week', 'Start_time', 'End_time', 'Length',
       'Name of show', 'Name of episode', 'Genre', 'First time or rerun',
       '# of episode in the season', 'Movie?',
       'Game of the Canadiens during episode?', 'Market Share_total',
       'Temperature in Montreal during episode'],
      dtype='object')

In [0]:
# first feature engineering
columns = ['Station', 'Channel Type', 'Season', 'Day of week', 'Length','Genre', 'First time or rerun',
       '# of episode in the season', 'Movie?',
       'Game of the Canadiens during episode?', 'Market Share_total']

In [21]:
for i, element in enumerate(columns):
  print (i, element)

0 Station
1 Channel Type
2 Season
3 Day of week
4 Length
5 Genre
6 First time or rerun
7 # of episode in the season
8 Movie?
9 Game of the Canadiens during episode?
10 Market Share_total


In [22]:
data.shape

(616656, 20)

In [23]:
# preprocessing
c = [element for i, element in enumerate(columns) if element != 'Market Share_total']
X = data[c]
Y = data['Market Share_total']
print("data shape: \t", data.shape, ", X shape: \t", X.shape, "\t, Y shape: \t", Y.shape)

data shape: 	 (616656, 20) , X shape: 	 (616656, 10) 	, Y shape: 	 (616656,)


In [24]:
#Label Encoder. In order to be able to get real classes back I didn't use for loop!
le0 = preprocessing.LabelEncoder()
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
le4 = preprocessing.LabelEncoder()
le5 = preprocessing.LabelEncoder()
le6 = preprocessing.LabelEncoder()
le7 = preprocessing.LabelEncoder()
le8 = preprocessing.LabelEncoder()
le9 = preprocessing.LabelEncoder()

X['Station'] = le0.fit_transform(X['Station'])
X['Channel Type'] = le1.fit_transform(X['Channel Type'])
X['Season'] = le2.fit_transform(X['Season'])
X['Day of week'] = le3.fit_transform(X['Day of week'])
X['Genre'] = le4.fit_transform(X['Genre'])
X['First time or rerun'] = le5.fit_transform(X['First time or rerun'])
X['# of episode in the season'] = le6.fit_transform(X['# of episode in the season'])
X['Movie?'] = le7.fit_transform(X['Movie?'])
X['Game of the Canadiens during episode?'] = le8.fit_transform(X['Game of the Canadiens during episode?'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [25]:
X.head()

,Station,Channel Type,Season,Day of week,Length,Genre,First time or rerun,# of episode in the season,Movie?,Game of the Canadiens during episode?
0,21,0,0,1,8,11,0,1,0,0
1,21,0,0,1,2,8,0,1,0,0
2,21,0,0,1,2,7,0,1,0,0
3,21,0,0,1,4,14,0,1,0,0
4,21,0,0,1,2,20,0,1,0,0


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [27]:
# simple multi layer percepton neural network...
est = make_pipeline(preprocessing.QuantileTransformer(),
                    MLPRegressor(hidden_layer_sizes=(50, 50),
                                 learning_rate_init=0.01,
                                 early_stopping=True))
est.fit(X_train, Y_train)
est.score(X_test, Y_test)

0.7096939335872228

In [0]:
poly_feature = preprocessing.PolynomialFeatures(degree=3, include_bias=False)
X_poly_tr = poly_feature.fit_transform(X_train)
X_poly_te = poly_feature.fit_transform(X_test)

In [34]:
# Try previous neural network with polynomial features
est = make_pipeline(preprocessing.QuantileTransformer(),
                    MLPRegressor(hidden_layer_sizes=(50, 50),
                                 learning_rate_init=0.01,
                                 early_stopping=True))
est.fit(X_poly_tr, Y_train)
est.score(X_poly_te, Y_test)

0.6893268843761424

In [36]:
# Linear Regression with polynomial features
lin_reg = LinearRegression().fit(X_poly_tr, Y_train)
lin_reg.score(X_poly_tr, Y_train)

0.586440026539283

In [33]:
model = Sequential([
    Dense(10, input_shape=(X_train.shape[1],)),
    Activation('relu'),
    Dense(50),
    Activation('linear'),
    Dense(1),
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=[metrics.mae, 'accuracy'])
model.fit(X_train, Y_train, epochs=50)

Epoch 1/50
431659/431659 [==============================] - 16s 38us/step - loss: 15.8997 - mean_absolute_error: 2.3586 - acc: 0.0244
Epoch 2/50
431659/431659 [==============================] - 16s 38us/step - loss: 14.9533 - mean_absolute_error: 2.2730 - acc: 0.0234
Epoch 3/50
431659/431659 [==============================] - 16s 38us/step - loss: 14.7219 - mean_absolute_error: 2.2453 - acc: 0.0225
Epoch 4/50
431659/431659 [==============================] - 17s 39us/step - loss: 14.4875 - mean_absolute_error: 2.2171 - acc: 0.0203
Epoch 5/50
431659/431659 [==============================] - 16s 37us/step - loss: 14.3789 - mean_absolute_error: 2.2023 - acc: 0.0205
Epoch 6/50
431659/431659 [==============================] - 16s 38us/step - loss: 14.3232 - mean_absolute_error: 2.1945 - acc: 0.0212
Epoch 7/50
431659/431659 [==============================] - 16s 36us/step - loss: 14.2884 - mean_absolute_error: 2.1914 - acc: 0.0218
Epoch 8/50
431659/431659 [==============================] - 16

In [28]:
score, acc = model.evaluate(X_test, Y_test)
print('Test score:', score)
print('Test accuracy:', acc)

184997/184997 [==============================] - 3s 16us/step
Test score: 12.53003053339373
Test accuracy: 2.1366952238317727


In [0]:
# Nan value
# Feature engineering
# try another algorithm
# cross validation